<a target="_blank" href="https://colab.research.google.com/github/giordamaug/HELP/blob/main/help/notebooks/prediction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://www.kaggle.com/notebooks/welcome?src=https://github.com/giordamaug/HELP/blob/main/help/notebooks/prediction.ipynb">
  <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/>
</a>

# Install HELP from GitHub
Skip this cell if you already have installed HELP.

In [ ]:
!pip install git+https://github.com/giordamaug/HELP.git

# Download the input files
In this cell we download from GitHub repository the label file and the attribute files. Skip this step if you already have these input files locally.

In [ ]:
tissue='Kidney'
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_HELP.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_BIO.csv
for i in range(5):
  !wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_CCcfs_{i}.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/help/datafinal/{tissue}_EmbN2V_128.csv

In [1]:
%cd ../datafinal

/Users/maurizio/HELP/help/datafinal


# Process the tissue attributes
In this code we load tissue gene attributes by several datafiles. We apply missing values fixing and data scaling with `sklearn.preprocessing.StandardScaler` on the `BIO` and `CCcfs` attributes, while no normalization and fixing on embedding attributes (`EmbN2V_128`). The attributes are all merged in one matrix by the `feature_assemble` function as input for the prediction model building.

In [20]:
tissue='Kidney'
import pandas as pd
from help.preprocess.loaders import feature_assemble_df
import os
df_y = pd.read_csv(f"{tissue}_HELP.csv", index_col=0)
df_y = df_y.replace({'aE': 'NE', 'sNE': 'NE'})    # E vs NE problem
#df_y = df_y[df_y['label'].isin(['aE']) == False]   # E vs sNE problem
#df_y = df_y[df_y['label'].isin(['E']) == False]    # aE vs sNE problem
#df_y = df_y[df_y['label'].isin(['sNE']) == False]  # E vs sE problem
print(df_y.value_counts(normalize=False))
features = [{'fname': f'{tissue}_BIO.csv', 'fixna' : False, 'normalize': 'std'},
            {'fname': f'{tissue}_CCcfs.csv', 'fixna' : False, 'normalize': 'std', 'nchunks' : 5},
            #{'fname': f'{tissue}_EmbN2V_128.csv', 'fixna' : False, 'normalize': None}
            ]
df_X, df_y = feature_assemble_df(df_y, features=features, verbose=True)
pd.merge(df_X, df_y, left_index=True, right_index=True, how='outer')

label
NE       16678
E         1253
Name: count, dtype: int64
Majority NE 16678 minority E 1253
[Kidney_BIO.csv] found 52532 Nan...
[Kidney_BIO.csv] Normalization with std ...


Loading file in chunks: 100%|██████████| 5/5 [00:02<00:00,  2.02it/s]


[Kidney_CCcfs.csv] found 6676644 Nan...
[Kidney_CCcfs.csv] Normalization with std ...
17236 labeled genes over a total of 17931
(17236, 3328) data input


,Gene length,Transcripts count,GC content,GTEX_kidney,Gene-Disease association,OncoDB_expression,HPA_kidney,GO-MF,GO-BP,GO-CC,...,Yolk plasma,Z chromosome,Z disc,Z granule,Zeta DNA polymerase complex,Zona pellucida receptor complex,Zonula adherens,Zymogen granule,Zymogen granule membrane,label
A1BG,0.003351,0.020942,0.501832,2.044542e-05,0.002950,NaN,0.000002,NaN,NaN,0.115385,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
A1CF,0.034865,0.047120,0.160530,1.980884e-05,NaN,0.556939,0.000232,0.069767,0.041026,0.096154,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
A2M,0.019624,0.062827,0.176932,3.377232e-03,0.073746,0.584540,0.005382,0.302326,0.056410,0.076923,...,0.0,0.000000,0.1176,0.0,0.0,0.0,0.0000,0.173014,0.0,NE
A2ML1,0.026017,0.041885,0.299948,5.123403e-07,0.017699,NaN,0.000000,0.069767,0.005128,0.038462,...,0.0,0.223561,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
A3GALT2,0.005784,0.000000,0.473739,1.421472e-06,NaN,0.663540,0.000000,0.069767,0.015385,0.057692,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.021209,0.010471,0.288257,7.073108e-06,NaN,0.634761,0.000055,NaN,NaN,0.000000,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
ZYG11B,0.040775,0.005236,0.248648,7.271294e-05,NaN,0.646090,0.000238,0.000000,0.005128,0.000000,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE
ZYX,0.003958,0.047120,0.539522,8.282866e-04,NaN,0.672638,0.000177,0.046512,0.035897,0.153846,...,0.0,0.000000,0.3982,0.0,0.0,0.0,0.2208,0.000000,0.0,NE
ZZEF1,0.056017,0.052356,0.304484,9.626291e-05,NaN,NaN,0.000121,0.093023,NaN,NaN,...,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000000,0.0,NE


# Prediction
We process k-fold cross validation of a LightGBM classifier (`n_splits=5`), and then we store predictions and print metrics.

In [22]:
from help.models.prediction import predict_cv_sv
df_scores, predictions = predict_cv_sv(df_X, df_y, n_voters=10, n_splits=5, balanced=True, verbose=True)
df_scores

/Users/maurizio/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Majority NE 15994, minority E 1242
{'E': 0, 'NE': 1}
label
NE       1600
E        1242
Name: count, dtype: int64
Classification with LGBM...


5-fold: 100%|██████████| 5/5 [00:14<00:00,  2.96s/it]


{'E': 0, 'NE': 1}
label
NE       1600
E        1242
Name: count, dtype: int64
Classification with LGBM...


5-fold: 100%|██████████| 5/5 [00:12<00:00,  2.43s/it]


...

{'E': 0, 'NE': 1}
label
NE       1599
E        1242
Name: count, dtype: int64
Classification with LGBM...


5-fold: 100%|██████████| 5/5 [00:14<00:00,  2.84s/it]


{'E': 0, 'NE': 1}
label
NE       1599
E        1242
Name: count, dtype: int64
Classification with LGBM...


5-fold: 100%|██████████| 5/5 [00:12<00:00,  2.43s/it]


,ROC-AUC,Accuracy,BA,Sensitivity,Specificity,MCC,CM
42,0.962853,0.90456,0.905131,0.905797,0.904464,0.580594,"[[1125, 117], [1528, 14466]]"


In [42]:
predictions.loc[np.intersect1d(csEGs, predictions.index.values)]

,label,prediction,probabilities
gene,,,
ACTG1,0,0,0.857604
ACTR6,0,0,0.733652
ARF4,0,0,0.582781
ARFRP1,0,1,0.033001
ARPC4,0,0,0.743110
CDK6,0,0,0.728350
CFLAR,0,1,0.151380
CHMP7,0,0,0.970504
COPS3,0,0,0.972604


## True Positive rates of context-specific EGs

In [48]:
import numpy as np
csEGs = pd.read_csv("csEG_Kidney.txt", index_col=0, header=None).index.values
indices = np.intersect1d(csEGs, predictions.index.values)
predictions = predictions.loc[indices]
num = len(predictions[predictions['label'] == predictions['prediction']])
den = len(predictions)
print(f"csEG Kidney TPR = {num /den:.3f} ({num}/{den})")

csEG Kidney TPR = 0.780 (46/59)
